In [ ]:
from typing import List
import copy
import pennylane as qml
import pennylane.numpy as np
from mitiq.zne.inference import RichardsonFactory
from mitiq.zne.scaling import fold_global
import logging
import qiskit

In [ ]:
SIMULATION_MODE = False    # running from local or real hardware
ibmqx_token="bec99ca4f96d63669f854d47e9e75da83bfb76a71f160a5b3e9cf7b9770448792d713cb6567119d286bc739703999a3c1ca98cd1b8f34f1cd3c39e317eaf5d87"

##  Build a simple noise model with depolarizing noise

In [ ]:
dev = qml.device('qiskit.ibmq', wires=2, backend='ibmq_qasm_simulator', ibmqx_token=ibmqx_token)

In [3]:
if SIMULATION_MODE:
    noise_strength = 0.3
    dev_noise_free = qml.device("default.mixed", wires=2)
    dev = qml.transforms.insert(
        dev_noise_free,
        qml.DepolarizingChannel,
        noise_strength
    )
elif IBMProvider.saved_accounts():
    qpu_device = Devices.Rigetti.AspenM3
    qpu_device_arn = qpu_device.value
    # provider = IBMProvider()
    # dev = qml.device(
    #     "qiskit.ibmq",
    #     wires=2,
    #     backend="ibmq_qasm_simulator",
    #     provider=provider
    # )
noise_less_dev = qml.device("default.qubit", wires=2)

In [4]:
def circuit():
    """
    @A circuit preparing a Bell state
    """
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

Zero-noise extrapolation (ZNE) is a noise mitigation technique. It works by intentionally scaling the noise of a quantum circuit to then extrapolate the zero-noise limit of an observable of interest. In this task, you will build a simple ZNE function from scratch:

- [x] Build a simple noise model with depolarizing noise 
- [x] Create different circuits to test your noise models and choose the observable to measure 
- [ ] Apply the unitary folding method. 
- [ ] Apply the extrapolation method to get the zero-noise limit. Different extrapolation methods achieve different results, such as Linear, polynomial, and exponential.
- [x] Compare mitigated and unmitigated results 
- [ ] Bonus: Run your ZNE function in real quantum hardware through the IBM Quantum Service

Check the Mitiq documentation for references. You are not allowed to use the functions from Mitiq or any other frameworks where ZNE is already implemented. 


## Applying unitary folding method

### Implement

In [5]:
def unitary_fold(circuit, scale_factor: int):
    # original ops
    circuit()
    original_ops = circuit.tape.operations
    ops = circuit.tape.copy(copy_operations=True).operations
    n, s = divmod(scale_factor - 1, 2)    
    # take all the gate, append and complex conjugate
    # For the 1st part  (U^H U)**n
    for i in range(n):
        for op in original_ops[::-1]:            
            ops.append(qml.adjoint(copy.copy(op)))

        for op in original_ops:            
            ops.append(op)
    
    # For the 2nd part (L_d^H .. L_s^H) (L_s .. L_d)    
    if s > 0:
        last_layers = original_ops[-s:]
        for op in last_layers[::-1]:
            ops.append(qml.adjoint(copy.copy(op)))
        for i in last_layers:
            ops.append(op)            

    # Return list of op to create the circuit
    return ops, circuit.tape.measurements


@qml.qnode(dev)
def circuit_from_ops(operations: List, measurements: List):    
    for op in operations:
        qml.apply(op)        
    return qml.apply(measurements[0])

NameError: name 'dev' is not defined

In [ ]:
device_circuit = qml.QNode(circuit, dev)
scale_factors = range(1,5)
results = []
for scale_factor in scale_factors:
    ops, measurements = unitary_fold(device_circuit, scale_factor)    
    results.append(circuit_from_ops(ops, measurements))    
    print(qml.draw(circuit_from_ops)(ops, measurements))
results

## Extrapolation

### Linear extrapolation

In [ ]:
def linear_extrapolation(x, y):
    opt_params = np.polyfit(x, y, 1)
    return opt_params[-1]

### Polinomial

In [ ]:
def polinomial_extraplation(x, y, order):
    opt_params = np.polyfit(x, y, order)
    return opt_params[-1]

### Exponential

In [ ]:
def exponential_extraplation(x, y):
    opt_params = np.polyfit(x, np.log(y), 1)
    return np.exp(opt_params[-1])

In [ ]:
extrapolate = {}
extrapolate['linear'] = linear_extrapolation(scale_factors, results)
extrapolate['polinomial'] = polinomial_extraplation(scale_factors, results, len(scale_factors)-1)
extrapolate['exp'] = exponential_extraplation(scale_factors, results)

## Create a reference ZNE mitigated circuit

In [ ]:
error_mitigated_device_circuit = qml.transforms.mitigate_with_zne(
    device_circuit,
    scale_factors,
    folding = fold_global,    
    extrapolate = RichardsonFactory.extrapolate,
)

## Compare the results between noisy circuit, reference method, and implemented method

In [ ]:
unmitigated = device_circuit()
mitigated = error_mitigated_device_circuit()
noise_free = qml.QNode(circuit, noise_less_dev)()
print(f"Unmitigated result {unmitigated}")
print(f"Noise free result  {noise_free}")
print()
print(f"Linear method: {extrapolate['linear']}")
print()
print("Polynomial method:")
print(f"\tReference mitigated result   {mitigated}")
print(f"\tImplemented mitigated result   {extrapolate['polinomial']}")
print()
print(f"Exp method: {extrapolate['exp']}")

In [ ]:
# todo track with pandas